In [2]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib as plt
from datetime import datetime, date, time

In [130]:
#file='../raw_data/202304.csv'
file='../raw_data/UK_real_estate_data.zip'
data=pd.read_csv(file,
                 delimiter="," ,
                 skipinitialspace=True,
                 usecols=[1,2,3,4,5,6,7,8,9,10,11,12,13],
                 names=['price', 'date', 'postcode','property_type', 'property_age', 'ground', 'number', 'additional_info', 'street', 'locality', 'town', 'district', 'county'],
                dtype={'price': np.int32, 'date': object, 'postcode': str, 'property_type': str,'property_age': str, 'ground': str, 'additional_info':str, 'street':str, 'locality':str, 'town':str, 'distric':str, 'county':str},
                compression='zip',
                #quoting=3,
                on_bad_lines='warn',
                date_format='%y-%m-%d %H:%M',
                na_values=["" ""],
                quotechar='"',
                doublequote=False,
                #nrows=10_000
)
#chunk = pd.read_csv(file, chunksize=50_000)
#chunk.sample(20)
data.head(10)

,price,date,postcode,property_type,property_age,ground,number,additional_info,street,locality,town,district,county
0,70000,1995-07-07 00:00,MK15 9HP,D,N,F,31,NaN,ALDRICH DRIVE,WILLEN,MILTON KEYNES,MILTON KEYNES,MILTON KEYNES
1,44500,1995-02-03 00:00,SR6 0AQ,T,N,F,50,NaN,HOWICK PARK,SUNDERLAND,SUNDERLAND,SUNDERLAND,TYNE AND WEAR
2,56500,1995-01-13 00:00,CO6 1SQ,T,N,F,19,NaN,BRICK KILN CLOSE,COGGESHALL,COLCHESTER,BRAINTREE,ESSEX
3,58000,1995-07-28 00:00,B90 4TG,T,N,F,37,NaN,RAINSBROOK DRIVE,SHIRLEY,SOLIHULL,SOLIHULL,WEST MIDLANDS
4,51000,1995-06-28 00:00,DY5 1SA,S,N,F,59,NaN,MERRY HILL,BRIERLEY HILL,BRIERLEY HILL,DUDLEY,WEST MIDLANDS
5,17000,1995-03-10 00:00,S65 1QJ,T,N,L,22,NaN,DENMAN STREET,ROTHERHAM,ROTHERHAM,ROTHERHAM,SOUTH YORKSHIRE
6,58000,1995-04-28 00:00,PE7 3AL,D,Y,F,4,NaN,BROOK LANE,FARCET,PETERBOROUGH,PETERBOROUGH,CAMBRIDGESHIRE
7,19500,1995-01-27 00:00,SK10 2QW,T,N,L,38,NaN,GARDEN STREET,MACCLESFIELD,MACCLESFIELD,MACCLESFIELD,CHESHIRE
8,20000,1995-01-16 00:00,SA6 5AY,D,N,F,592,NaN,CLYDACH ROAD,YNYSTAWE,SWANSEA,SWANSEA,SWANSEA
9,137500,1995-03-31 00:00,NR2 2NQ,D,N,F,26,NaN,LIME TREE ROAD,NORWICH,NORWICH,NORWICH,NORFOLK


In [131]:
# Only select properties within Greater London
df=data[data['county']=='GREATER LONDON']

# after, save this to new csv and work with this one.

In [133]:
df.dtypes
df.shape


(3623319, 13)

In [126]:
# dt = datetime.strptime("21/11/06 16:30", "%d/%m/%y %H:%M")
#data.date=data.date.apply(lambda x: datetime.strptime(x, '%y-%d-%m %H:%M'))

In [ ]:
# Convert date to datetime and create separate columns with month,year,day
df.date=pd.to_datetime(df['date'])
df['year']=df.loc['date'].apply(lambda x: x.year).astype(np.int16)
df['month']=df.loc['date'].apply(lambda x: x.month).astype(np.int16)
df['day']=df.loc['date'].apply(lambda x: x.day).astype(np.int16)

In [138]:
df.columns


Index(['price', 'date', 'postcode', 'property_type', 'property_age', 'ground',
       'number', 'additional_info', 'street', 'locality', 'town', 'district',
       'county', 'year', 'month', 'day'],
      dtype='object')

In [143]:
df.to_csv('../raw_data/london_real_estate_data.zip',
          doublequote=False,
          date_format='%y-%m-%d %H:%M',
          index=False)

In [149]:
tmp=pd.read_csv('../raw_data/london_real_estate_data.zip', compression='zip',
                dtype={'price': np.int32,'day':np.int16, 'month':np.int16,'year':np.int16}
)
tmp.head(10)

,price,date,postcode,property_type,property_age,ground,number,additional_info,street,locality,town,district,county,year,month,day
0,62750,95-10-02 00:00,BR3 4AT,T,N,F,56,NaN,EDEN ROAD,BECKENHAM,BECKENHAM,BROMLEY,GREATER LONDON,1995,10,2
1,78000,95-09-29 00:00,SW16 3BL,S,N,F,208,NaN,GREEN LANE,LONDON,LONDON,CROYDON,GREATER LONDON,1995,9,29
2,51000,95-10-31 00:00,UB5 5LH,S,N,F,57,NaN,BENGARTH ROAD,NORTHOLT,NORTHOLT,EALING,GREATER LONDON,1995,10,31
3,405000,95-04-07 00:00,N1 2JU,S,N,F,49,NaN,CANONBURY PARK NORTH,LONDON,LONDON,ISLINGTON,GREATER LONDON,1995,4,7
4,60000,95-12-21 00:00,IG1 1PP,T,N,F,209,NaN,HAMPTON ROAD,ILFORD,ILFORD,REDBRIDGE,GREATER LONDON,1995,12,21
5,67000,95-04-24 00:00,SW17 0DF,T,N,F,80,NaN,KENLOR ROAD,LONDON,LONDON,WANDSWORTH,GREATER LONDON,1995,4,24
6,100000,95-04-27 00:00,N3 2SG,S,N,F,184,NaN,STATION ROAD,LONDON,LONDON,BARNET,GREATER LONDON,1995,4,27
7,90000,95-07-26 00:00,N7 8AD,T,N,F,49,NaN,WESTBOURNE ROAD,LONDON,LONDON,ISLINGTON,GREATER LONDON,1995,7,26
8,89250,95-09-19 00:00,N7 7PS,T,N,F,36B,NaN,BERRIMAN ROAD,NaN,LONDON,ISLINGTON,GREATER LONDON,1995,9,19
9,180000,95-06-01 00:00,SM2 5JB,D,N,F,104,NaN,UPLAND ROAD,SUTTON,SUTTON,SUTTON,GREATER LONDON,1995,6,1


In [150]:
tmp.dtypes

price               int32
date               object
postcode           object
property_type      object
property_age       object
ground             object
number             object
additional_info    object
street             object
locality           object
town               object
district           object
county             object
year                int16
month               int16
day                 int16
dtype: object